## Naive Bayes Practice

### 직접 구현

- 긍정(1), 부정(0)
- 새로운 메일이 왔는데, 'happy','weekend'가 포함되어 있을 경우, 이 메일은 긍정인가, 부정인가?


- 내 코드

In [117]:
import pandas as pd
import numpy as np
file= pd.read_csv("naivebayes_example.csv") # 읽어오기
file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [118]:
doc_ls=[]
for doc in file['mail']:
    doc_ls.append(doc.split(' '))
doc_ls

[['i', 'love', 'you'],
 ['love', 'happy', 'weekend'],
 ['bore', 'work', 'job'],
 ['i', 'hate', 'you'],
 ['bore', 'weekend'],
 ['happy', 'together']]

In [119]:
from collections import defaultdict

word2id = defaultdict(lambda : len(word2id))

for doc in doc_ls:
    for token in doc:
        word2id[token]
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'bore': 5,
             'work': 6,
             'job': 7,
             'hate': 8,
             'together': 9})

In [120]:
positive=[]
negative=[]
doc_ls=[]
for i,sen in enumerate(file['mail']):
    doc_ls.append(sen.split(' '))
for i,tog in enumerate(doc_ls):
    for word in tog:
         if file['label'][i] == 1:
            positive.append(word)
         else:
            negative.append(word)
print('positive',positive)
print('negative',negative)

positive ['i', 'love', 'you', 'love', 'happy', 'weekend', 'happy', 'together']
negative ['bore', 'work', 'job', 'i', 'hate', 'you', 'bore', 'weekend']


In [121]:
pos = np.zeros(len(word2id), dtype=int)
for i,doc in enumerate(positive):
    pos[word2id[doc]] += 1 #해당 토큰의 위치(column)
print(pos)

[1 2 1 2 1 0 0 0 0 1]


In [122]:
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'bore': 5,
             'work': 6,
             'job': 7,
             'hate': 8,
             'together': 9})

In [123]:
df= pd.DataFrame(index=word2id)
df

""
i
love
you
happy
weekend
bore
work
job
hate
together


In [124]:
df['Positive']= pos
df

,Positive
i,1
love,2
you,1
happy,2
weekend,1
bore,0
work,0
job,0
hate,0
together,1


In [125]:
neg = np.zeros(len(word2id), dtype=int)
for i,doc in enumerate(negative):
    neg[word2id[doc]] += 1 #해당 토큰의 위치(column)
print(neg)

[1 0 1 0 1 2 1 1 1 0]


In [126]:
df['Negative']= neg
df

,Positive,Negative
i,1,1
love,2,0
you,1,1
happy,2,0
weekend,1,1
bore,0,2
work,0,1
job,0,1
hate,0,1
together,1,0


In [127]:
df['Positive'].sum()

8

In [141]:
total_token=len(word2id)
P_positive=df['Positive'].sum()
for i in range(total_token):
    P_w_positive=(df['Positive'][i]+1)/(P_positive+total_token)
    df['P(w|positive)'][i]=P_w_positive

df

C:\Users\user\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Positive,Negative,P(w|negative),P(w|positive)
i,1,1,0.111111,0.111111
love,2,0,0.055556,0.166667
you,1,1,0.111111,0.111111
happy,2,0,0.055556,0.166667
weekend,1,1,0.111111,0.111111
bore,0,2,0.166667,0.055556
work,0,1,0.111111,0.055556
job,0,1,0.111111,0.055556
hate,0,1,0.111111,0.055556
together,1,0,0.055556,0.111111


In [138]:
total_token=len(word2id)
P_negative=df['Negative'].sum()
for i in range(total_token):
    P_w_negative=(df['Negative'][i]+1)/(P_negative+total_token)
    df["P(w|negative)"][i]=P_w_negative


C:\Users\user\anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [142]:
df

,Positive,Negative,P(w|negative),P(w|positive)
i,1,1,0.111111,0.111111
love,2,0,0.055556,0.166667
you,1,1,0.111111,0.111111
happy,2,0,0.055556,0.166667
weekend,1,1,0.111111,0.111111
bore,0,2,0.166667,0.055556
work,0,1,0.111111,0.055556
job,0,1,0.111111,0.055556
hate,0,1,0.111111,0.055556
together,1,0,0.055556,0.111111


In [147]:
import math
# P(positive|happy,weekend)
P_positive_1= (P_positive/(P_negative+P_positive))
P_negative_1= (P_negative/(P_negative+P_positive))
son= math.exp(math.log10(df['P(w|positive)']['happy']*df['P(w|positive)']['weekend']*P_positive_1))
son1= math.exp(math.log10(df['P(w|negative)']['happy']*df['P(w|negative)']['weekend']*P_negative_1))


In [149]:
P_positive_happy_weekend = son/(son+son1)
P_negative_happy_weekend = son1/(son1+son)
print(P_positive_happy_weekend)
print(P_negative_happy_weekend)

0.617067870962967
0.382932129037033


## Using Sklearn

In [131]:
import pandas as pd
import numpy as np
file= pd.read_csv("naivebayes_example.csv") # 읽어오기
file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [132]:
X_train = list(file['mail'])
Y_train = list(file['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [133]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# CountVectorizer 선언
count_vect = CountVectorizer()

# fit and transform
X_train_counts = count_vect.fit_transform(X_train)

#MultinomialNB 선언
clf= MultinomialNB().fit(X_train_counts, Y_train)

In [134]:
#예측
print(clf.predict(count_vect.transform(['happy weekend'])))

#확률
print(clf.predict_proba(count_vect.transform(['happy weekend'])))

[1]
[[0.25 0.75]]


## 선생님 풀이

In [151]:
import pandas as pd
import numpy as np
input_file= pd.read_csv("naivebayes_example.csv") # 읽어오기
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [152]:
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


### 토큰 빈도수 및 문서별 토큰수 계산
- 확률계산을 위한 준비

In [155]:
# 토큰별로 문서 내 빈도수 카운팅

from collections import defaultdict
wordfreq = defaultdict(lambda: [0,0]) # lambda: 새로운 단어가 추가 될 때 default 를 [0,0 ]으로 하겠다
for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:
            wordfreq[word][0] += 1 # 긍정인 경우 0번째 인덱스에 1 더하기
        else: 
            wordfreq[word][1] += 1 # 부정인 경우 1번째 인덱스에 1 더하기
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [157]:
# 긍정, 부정 빈도수 계산
긍정전체토큰수=[]
부정전체토큰수=[]
for key, (cnt1, cnt0) in wordfreq.items():
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)

전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정, 전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


### Training: 토큰별 조건부 확률 계산

In [159]:
wordprobs = defaultdict(lambda: [0,0])
for key, (cnt1, cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1+1)/(전체갯수_긍정+ len(wordfreq))
    wordprobs[key][1] = (cnt0+1)/(전체갯수_긍정+ len(wordfreq))
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

### Classify: 신규텍스트가 주어졌을 때 확률 계산

In [162]:
import math
doc = "happy weekend"
tokens = doc.split()
tokens

['happy', 'weekend']

In [165]:
#초기 값을 모두 0으로 처리
log_prob1= log_prob0= 0.0

#모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0)
log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_긍정+전체갯수_부정))

prob1 = math.exp(log_prob1)
print("prob1",prob1)

prob0 = math.exp(log_prob0)
print("prob0",prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정 확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))

prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정 확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%
